---
## Import packages

In [1]:
# !pip install farasapy
# !pip install Tashaphyne==0.3.4.1
# !pip install wordcloud
# !pip install ar_wordcloud
# import nltk
# nltk.download('stopwords')
# nltk.download('punkt')

     |████████████████████████████████| 244 kB 691 kB/s eta 0:00:01
     |████████████████████████████████| 126 kB 2.8 MB/s eta 0:00:01
     |████████████████████████████████| 220 kB 903 kB/s eta 0:00:01
  Created wheel for wordcloud: filename=wordcloud-1.8.1-cp38-cp38-macosx_10_9_x86_64.whl size=158803 sha256=d6b7e89b393619c9b97e4f68124c1f15e26ad78d012d2209cf27cc51be05a695
  Stored in directory: /Users/amalalthaqafi/Library/Caches/pip/wheels/4d/3f/0d/a2ba9b7895c9f1be89018b3141c3df3d4f9c786c882ccfbc3b
Successfully built wordcloud
  Attempting uninstall: wordcloud
    Found existing installation: wordcloud 1.6.0
    Uninstalling wordcloud-1.6.0:
      Successfully uninstalled wordcloud-1.6.0


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/amalalthaqafi/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/amalalthaqafi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

import matplotlib as mpl
import matplotlib.pyplot as plt

import string

import re

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
# from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report

from ar_wordcloud import ArabicWordCloud

In [3]:
from tashaphyne.stemming import ArabicLightStemmer

In [4]:
import nltk
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.stem.isri import ISRIStemmer

In [5]:
from farasa.pos import FarasaPOSTagger
from farasa.ner import FarasaNamedEntityRecognizer
from farasa.diacratizer import FarasaDiacritizer
from farasa.segmenter import FarasaSegmenter
from farasa.stemmer import FarasaStemmer

---
## Read and exploring the data

In [8]:
df = pd.read_csv('SaudiNews')
df.drop(columns= ['Unnamed: 0'], inplace= True)
df

,source,title,content
0,aleqtisadiya,هيئة المسح الجيولوجي الأمريكية: زلزال بقوة 6.3...,قالت هيئة المسح الجيولوجي الأمريكيةإن زلزالا ...
1,alweeam,توفر وظائف أكاديمية بجامعــة الملك فيصل,أعلنت جامعة الملك فيصل بالأحساء توفر عدد من ا...
2,sabq,&quot;أدبي جدة&quot; يدشن دورة &quot;مفاهيم كت...,سبق- جدة: دشَّن أدبي جدة اللقاء الأول من دورة...
3,alyaum,قتلة الطموح,«في مجتمعنا مبدأ يمنع تطور الفرد فكرياً واجتم...
4,alriyadh,دراسة مسببات عدم انخفاض السلع في المملكة بانخف...,أكد الدكتور سليمان السماحي الرئيس التنفيذي...
...,...,...,...
9995,okaz,تطويق العنف المدرسي ببرنامج «رفق» الإرشادي,تعتزم وزارة التعليم، في مستهل العام الدراسي ا...
9996,aawsat,ولي ولي العهد السعودي يتقبل بيعة منسوبي وزارة...,استقبل الأمير محمد بن سلمان بن عبد العزيز ولي ...
9997,alriyadh,هبوط أول طائرة عسكرية سعودية في مطار عدن بعد د...,هبطت طائرة عسكرية سعودية في مطار مدينة عدن...
9998,alwatan,"استثمار ""الربيعة"" عالميا",هناك دول في العالم تنفق مع شركات علاقات عالمي...


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   source   10000 non-null  object
 1   title    9978 non-null   object
 2   content  9964 non-null   object
dtypes: object(3)
memory usage: 234.5+ KB


In [10]:
df.duplicated().sum()

94

In [11]:
df.drop_duplicates(inplace = True)

---
## NLP Preprocessing

In [12]:
max(df.content.str.len())

26852.0

In [13]:
min(df.content.str.len())

1.0

**removing observtions less than 120**

In [14]:
df = df[df.content.str.len() > 120]

In [15]:
df.shape

(9716, 3)

In [16]:
df.content.isna().sum()

0

### **Removing Non-arabic letters, numbers, amiss typo and special characters**

In [17]:
# function to clean and normalize text 
def clean_text(text):
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ']  
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", text)
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
        
    text = text.strip()
    
    return text

In [18]:
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)


def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    
    return text


def remove_diacritics(text):
    text = re.sub(arabic_diacritics, '', text)
    return text


def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)


def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

In [19]:
# clean and normalize text
df['content'] = df.content.apply(lambda x: clean_text(x))

# remove punctuation
df['content'] = df.content.apply(lambda x: remove_punctuations(x))

# remove diacritics
df['content'] = df.content.apply(lambda x: remove_diacritics(x))

# remove repeating char
df['content'] = df.content.apply(lambda x: remove_repeating_char(x))

# remove english letters
df.content = df.content.apply(lambda x: re.sub(r'[a-zA-Z]', '', x))

# remove a special character _
df.content = df.content.apply(lambda x: re.sub(r'[_]+', '', x))

# remove space
df['content'] = df.content.str.replace('\d+', '')

In [20]:
df

,source,title,content
0,aleqtisadiya,هيئة المسح الجيولوجي الأمريكية: زلزال بقوة 6.3...,قالت هيئه المسح الجيولوجي الامريكيهان زلزالا ب...
1,alweeam,توفر وظائف أكاديمية بجامعــة الملك فيصل,اعلنت جامعه الملك فيصل بالاحساء توفر عد من الو...
2,sabq,&quot;أدبي جدة&quot; يدشن دورة &quot;مفاهيم كت...,سبق جده دشن ادبي جده القاء الاول من دوره مفاهي...
3,alyaum,قتلة الطموح,«في مجتمعنا مبدا يمنع تطور الفرد فكريا واجتماع...
4,alriyadh,دراسة مسببات عدم انخفاض السلع في المملكة بانخف...,اكد الدكتور سليمان السماحي الرئيس التنفيذي لجم...
...,...,...,...
9995,okaz,تطويق العنف المدرسي ببرنامج «رفق» الإرشادي,تعتزم وزاره التعليم في مستهل العام الدراسي الم...
9996,aawsat,ولي ولي العهد السعودي يتقبل بيعة منسوبي وزارة...,استقبل الامير محمد بن سلمان بن عبد العزيز ولي ...
9997,alriyadh,هبوط أول طائرة عسكرية سعودية في مطار عدن بعد د...,هبطت طائره عسكريه سعوديه في مطار مدينه عدن جنو...
9998,alwatan,"استثمار ""الربيعة"" عالميا",هناك دول في العالم تنفق مع شركات علاقات عالميه...


In [21]:
# Convert to list
data = df[df.columns[2]].values.tolist()

### **Stemming**

In [22]:
farasa_st = FarasaStemmer()

[2021-12-27 19:27:56,104 - farasapy_logger - ERROR]: error occured: Command '['java', '-version']' returned non-zero exit status 1..


Exception: We could not check for java version on the machine. Please make sure you have installed Java 1.7+ and add it to your PATH.

In [ ]:
# st = ISRIStemmer()
# ArListem = ArabicLightStemmer()

# def stemSentence(sentence):
#     token_words=word_tokenize(sentence)
#     stem_sentence=[]
#     for word in token_words:
#         stem_sentence.append(farasa_st.stem(word))
#         #stem_sentence.append(ArListem.light_stem(word))
#         stem_sentence.append(' ')
#     return "".join(stem_sentence)

# df.content = df.content.apply(lambda x: stemSentence(x))

### **TF-IDF Vectorizer**

In [ ]:
# the document-term matrix 
arb_stopwords = set(nltk.corpus.stopwords.words('arabic'))
tfidf = TfidfVectorizer(stop_words=arb_stopwords)
doc_words = tfidf.fit_transform(df.content)
pd.DataFrame(doc_words.toarray(),columns=tfidf.get_feature_names())

In [ ]:
# # To display full text
# pd.set_option('display.max_colwidth',-1)#can see the whole data on one single row
# complaints_df

## EDA

In [ ]:
# print WordCloud visualization

awc = ArabicWordCloud(background_color="black",)

mpl.rcParams['figure.figsize']=(20,12.0)  
mpl.rcParams['font.size']=12            
mpl.rcParams['savefig.dpi']=100             
mpl.rcParams['figure.subplot.bottom']=.1 

# wordcloud = awc.generate(str(tfidf.get_feature_names())).from_text(
#     str(tfidf.get_feature_names()))

wordcloud = awc.generate(str(tfidf.get_feature_names())).from_text(str(tfidf.get_feature_names()))

print(wordcloud)
fig = plt.figure(1)
plt.imshow(wordcloud.recolor(colormap='Dark2'),interpolation='bilinear')
#plt.title("Word Cloud of all the words")
plt.axis('off')